In [1]:
pip install pandas scikit-learn numpy dandelion-optimization


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement dandelion-optimization (from versions: none)
ERROR: No matching distribution found for dandelion-optimization


In [2]:
pip install pandas scikit-learn numpy dandelion-optimization

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement dandelion-optimization (from versions: none)
ERROR: No matching distribution found for dandelion-optimization


In [3]:
import os
print(os.getcwd())

C:\Users\sakha


In [4]:
import os
print(os.listdir())

['.altera.sbt4e', '.bash_history', '.cache', '.conda', '.condarc', '.continuum', '.eclipse', '.gitconfig', '.ipynb_checkpoints', '.ipython', '.jupyter', '.lesshst', '.matplotlib', '.node_repl_history', '.p2', '.packettracer', '.viminfo', '.VirtualBox', '.vscode', 'anaconda3', 'AppData', 'Application Data', 'Applied Machine Learning in Python.ipynb', 'Cisco Packet Tracer 8.2.2', 'Coding.cpp', 'Contacts', 'Cookies', 'CSE-1021.ipynb', 'Dandelion_Optimizer.ipynb', 'Desktop', 'Documents', 'Downloads', 'eclipse', 'Favorites', 'hsKfLshPLQ9', 'IntelGraphicsProfiles', 'Lab Activity - 1.ipynb', 'Lab Activity - 2.ipynb', 'Lab Activity 1.ipynb', 'Links', 'Local Settings', 'Medicine_description.csv', 'Music', 'My Documents', 'NetHood', 'NPProt.bkp', 'NPProt.exe', 'NTUSER.DAT', 'ntuser.dat.LOG1', 'ntuser.dat.LOG2', 'NTUSER.DAT{67913727-e4a2-11ef-ae27-cb194df4ebc5}.TxR.0.regtrans-ms', 'NTUSER.DAT{67913727-e4a2-11ef-ae27-cb194df4ebc5}.TxR.1.regtrans-ms', 'NTUSER.DAT{67913727-e4a2-11ef-ae27-cb194df4ebc

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, mean_squared_error

In [6]:
# Load Dataset
data  = pd.read_csv("Medicine_description.csv")

# Show the first few rows
data .head()

,Drug_Name,Reason,Description
0,A CN Gel(Topical) 20gmA CN Soap 75gm,Acne,Mild to moderate acne (spots)
1,A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...,Acne,A RET 0.025% is a prescription medicine that i...
2,ACGEL CL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
3,ACGEL NANO Gel 15gm,Acne,It is used to treat acne vulgaris in people 12...
4,Acleen 1% Lotion 25ml,Acne,treat the most severe form of acne (nodular ac...


In [7]:
print(data.columns)

Index(['Drug_Name', 'Reason', 'Description'], dtype='object')


In [8]:
# Data Preprocessing
data.dropna(inplace=True)  # Remove missing values
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(data['Description']).toarray()
y = data['Reason'].astype('category').cat.codes  # Convert target to numeric

In [9]:
# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Dandelion Optimizer (DO) Implementation
class DandelionOptimizer:
    def __init__(self, pop_size, max_iter, dim, fitness_function):
        self.pop_size = pop_size
        self.max_iter = max_iter
        self.dim = dim
        self.fitness_function = fitness_function
        self.population = np.random.rand(pop_size, dim)
        self.best_solution = None
        self.best_fitness = float('inf')
    
    def optimize(self):
        for t in range(self.max_iter):
            fitness_values = np.array([self.fitness_function(ind) for ind in self.population])
            elite_index = np.argmin(fitness_values)
            X_elite = self.population[elite_index]
            
            for i in range(self.pop_size):
                if np.random.rand() < 0.5:
                    alpha = np.random.rand() * ((1 / self.max_iter**2) * t**2 - (2 / self.max_iter) * t + 1)
                    self.population[i] = self.population[i] + alpha * np.random.randn(self.dim)
                else:
                    k = 1 - np.random.rand() * (1 / (self.max_iter**2 - 2 * self.max_iter + 1) * t**2 - (2 / (self.max_iter**2 - 2 * self.max_iter + 1)) * t + 1)
                    self.population[i] = self.population[i] * k
                
            self.population = np.clip(self.population, 0, 1)  # Keep within bounds
            
            if fitness_values[elite_index] < self.best_fitness:
                self.best_solution = X_elite
                self.best_fitness = fitness_values[elite_index]
        
        return self.best_solution, self.best_fitness


In [11]:
# Fitness Function (Random Forest Accuracy)
def fitness_function(params):
    n_estimators = int(params[0] * 200) + 1
    max_depth = int(params[1] * 20) + 1
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return -accuracy_score(y_test, preds)  # Negative for minimization


In [13]:
# Run Optimization

print("Starting Optimization...")
do = DandelionOptimizer(pop_size=10, max_iter=20, dim=2, fitness_function=fitness_function)
best_params, best_fitness = do.optimize()
print(f"Optimization Finished. Best Parameters: {best_params}, Best Fitness: {best_fitness}")


Starting Optimization...
Optimization Finished. Best Parameters: [0.08020142 0.32865051], Best Fitness: -0.8796172674677347


In [16]:
# Train Optimized Model
rf_optimized = RandomForestClassifier(n_estimators=int(best_params[0] * 200) + 1,
                                      max_depth=int(best_params[1] * 20) + 1,
                                      random_state=42)
rf_optimized.fit(X_train, y_train)
y_pred = rf_optimized.predict(X_test)

In [17]:
rf_optimized = RandomForestClassifier(n_estimators=int(best_params[0] * 200) + 1,
                                      max_depth=int(best_params[1] * 20) + 1,
                                      random_state=42,
                                      class_weight='balanced')

In [18]:
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)


In [19]:
# Evaluation Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

# Print Results
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1-score: {f1:.4f}')
print(f'MSE: {mse:.4f}')
print(f'RMSE: {rmse:.4f}')


Accuracy: 0.6451
Precision: 0.6167
Recall: 0.6451
F1-score: 0.5885
MSE: 8.6611
RMSE: 2.9430


C:\Users\sakha\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
